# $c_6$-dependence of $gg(\rightarrow h^{\ast})\rightarrow ZZ$ process

In [2]:
# modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# helper functions

def solve_polynomial_coefficients(x, y):
  """
  Finds the coefficients of the polynomial that fits the given (x, y) coordinates.

  Parameters:
  x (array-like): An array of x coordinates.
  y (array-like): An array of y coordinates.

  Returns:
  numpy.ndarray: The coefficients of the polynomial.
  """
  # Ensure x and y are numpy arrays
  x = np.array(x)
  y = np.array(y)
  
  # Number of points
  N = len(x) - 1
  
  # Construct the Vandermonde matrix
  V = np.vander(x, N+1, increasing=True)
  
  # Solve for the polynomial coefficients
  coefficients = np.linalg.solve(V, y)
  
  return coefficients

def polynomial(coeffs):
  """
  Construct a polynomial over a domain.

  Parameters:
  coeffs (array-like): polynomial coefficients.

  Returns:
  Callable: Invoke with the the domain to get values of the polynomial function over it.
  """
  def polynomial(x):
    return sum(c * x**i for i, c in enumerate(coeffs))
  return polynomial

In [4]:
# read events of each sample
filepath = '/raven/u/taepa/mcfm/MCFM-10.3/Bin/ggZZ2e2m_all/events.csv'
events = pd.read_csv(filepath, nrows=1000)

xsec = 1.4787447,
lumi = 3000.0
events['wt'] = events['wt'] * xsec * lumi / np.sum(events['wt'])

## Check: component-wise $c_6$-dependence factorization

$$ p(x_i | \theta) = \frac{1}{\nu(\theta)} \sum_{j} f_j(x_i | \theta) \cdot p(x_i | \theta_0) $$

In [14]:
random_event_indices = [1,2,3,123,321]

print([events['msq_sig_c6_21'][random_event_index] / events['msq_sig_sm'][random_event_index] for random_event_index in random_event_indices])

print([events['msq_sbi_c6_21'][random_event_index] / events['msq_sbi_sm'][random_event_index] for random_event_index in random_event_indices])

[np.float64(0.7893807702998114), np.float64(0.7210032761263563), np.float64(0.9387082794942915), np.float64(0.7652391644233368), np.float64(0.8384089397427711)]
[np.float64(1.0019470520246236), np.float64(1.0042125345351052), np.float64(1.010898080763965), np.float64(1.0015517180306666), np.float64(1.014124917320088)]
